### "Harmonic NDVI Time Series Clustering with Python and GEE"
Joao Otavio Nascimento Firigato

https://joaootavionf007.medium.com/harmonic-ndvi-time-series-clustering-with-python-and-gee-f5c1a8b47f6f

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [2]:
import folium
from folium import plugins
from IPython.display import Image
import geopandas as gpd
import json
print(folium.__version__)
from ipygee import*
import math
import pandas as pd
from tslearn.clustering import TimeSeriesKMeans
from tslearn.utils import to_time_series_dataset

0.12.1.post1


In [3]:
AOI =  ee.Geometry.Polygon(
[[[-56.37397887990624, -12.737207954893526],
[-56.37397887990624, -13.300834158918619],
[-55.37113311574608, -13.300834158918619],
[-55.37113311574608, -12.737207954893526]]])
points = ee.FeatureCollection.randomPoints(AOI,100)

In [4]:
months = ee.List.sequence(1,12)
years = ee.List.sequence(2016, 2019)

In [5]:
MD_NDVI = ee.ImageCollection('MODIS/MOD09GA_006_NDVI').filterDate('2016-1-1','2019-12-31').filterBounds(AOI).select('NDVI')
modis_ndvi = MD_NDVI.median().clip(AOI)
mean_ndvi = MD_NDVI.mean().clip(AOI)

In [6]:
vis_params = {'min': 0, 'max': 1, 'palette': ['red', 'yellow','green']}

basemaps = {
'Google Maps': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Maps',
overlay = True,
control = True
),
'Google Satellite': folium.TileLayer(
tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
attr = 'Google',
name = 'Google Satellite',
overlay = True,
control = True
)}

In [7]:
def add_ee_layer(self, ee_object, vis_params, name):
    try:
        if isinstance(ee_object, ee.image.Image):
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name, 
            overlay = True,
            control = True
            ).add_to(self)

        elif isinstance(ee_object, ee.geometry.Geometry):
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        elif isinstance(ee_object,ee.featurecollection.FeatureCollection):
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

    except:
        print("Could not display {}".format(name))

In [8]:
folium.Map.add_ee_layer = add_ee_layer
my_map = folium.Map(location=[-13.0912068,-55.9881647], zoom_start=10)
basemaps['Google Maps'].add_to(my_map)
my_map.add_ee_layer(modis_ndvi, vis_params, 'NDVI')
my_map.add_ee_layer(points.geometry(), {}, 'Points')
my_map.add_child(folium.LayerControl())
display(my_map)

In [9]:
def monthly(collection):
    img_coll = ee.ImageCollection([])
    for y in years.getInfo():
        for m in months.getInfo():
            filtered = collection.filter(ee.Filter.calendarRange(y,y,'year')).filter(ee.Filter.calendarRange(m, m, 'month'))
            filtered = filtered.median()
            img_coll = img_coll.merge(filtered.set('year', y).set('month', m)
                                      .set('system:time_start', ee.Date.fromYMD(y, m, 1).getInfo()['value']))
    return img_coll

Monthly_MD = monthly(MD_NDVI)

In [10]:
Point_1 = ee.Geometry.Point([-56.1070717726807,-13.168187045136754])
MD_ndvi = chart.Image.series(**{'imageCollection': Monthly_MD,
'region': Point_1,
'reducer': ee.Reducer.mean(),
'scale': 250,
'xProperty': 'system:time_start'})
MD_ndvi.renderWidget(width='50%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [11]:
dependent = 'NDVI'
harmonics = 3
harmonicFrequencies = list(range(1, harmonics+1))

def getNames (base, lst_freq):
    name_lst = []
    for i in lst_freq:
        name_lst.append(ee.String(base + str(i)))
    return name_lst

cosNames = getNames('cos_', harmonicFrequencies)
sinNames = getNames('sin_', harmonicFrequencies)
independents = ee.List(['constant','t']).cat(cosNames).cat(sinNames)

In [12]:
def addConstant (image):
    return image.addBands(ee.Image(1))

def addTime (image):
    date = ee.Date(image.get('system:time_start'))
    years = date.difference(ee.Date('1970-01-01'), 'year')
    timeRadians = ee.Image(years.multiply(2 * math.pi))
    return image.addBands(timeRadians.rename('t').float())

def addHarmonics (image):
    frequencies = ee.Image.constant(harmonicFrequencies)
    time = ee.Image(image).select('t')
    cosines = time.multiply(frequencies).cos().rename(cosNames)
    sines = time.multiply(frequencies).sin().rename(sinNames)
    return image.addBands(cosines).addBands(sines)

harmonicMODIS2 = Monthly_MD.map(addConstant).map(addTime).map(addHarmonics)

harmonicTrend = harmonicMODIS2.select(independents.add(dependent)).reduce(ee.Reducer.linearRegression(independents.length(), 1))
    
harmonicTrendCoefficients = harmonicTrend.select('coefficients').arrayProject([0]).arrayFlatten([independents])
        
fittedHarmonic = harmonicMODIS2.map(lambda image: image.addBands(image.select(independents)
                                                       .multiply(harmonicTrendCoefficients)
                                                       .reduce('sum')
                                                       .rename('fitted')))

In [13]:
MD_ndvi = chart.Image.series(**{'imageCollection': fittedHarmonic,  'region': Point_1,
'reducer': ee.Reducer.mean(),
'bands' : 'fitted',
'scale': 250,
'xProperty': 'system:time_start'})
MD_ndvi.renderWidget(width='50%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [14]:
df_points = pd.DataFrame([])

for n in range(1,points.size().getInfo() + 1):
    feat = ee.Geometry.Point(points.getInfo()["features"][n-1]['geometry']['coordinates'])
    point_ndvi = chart.Image.series(**{
    'imageCollection': fittedHarmonic,
    'region': feat,
    'reducer': ee.Reducer.mean(),
    'bands' : 'fitted',
    'scale': 250,
    'xProperty': 'system:time_start'})
    df = point_ndvi.dataframe
    df = df.transpose().copy()
    df_points = df_points.append(df, ignore_index=True)
    
X = df_points.values
formatted_X = to_time_series_dataset(X)
print(formatted_X.shape)
km = TimeSeriesKMeans(n_clusters=3, metric="softdtw")
labels = km.fit_predict(formatted_X)
newdata = gpd.GeoDataFrame.from_features(points.getInfo()["features"])
newdata['Class'] = labels

(100, 48, 1)


In [15]:
phase = harmonicTrendCoefficients.select('sin_1').atan2(harmonicTrendCoefficients.select('cos_1')).unitScale(-math.pi, math.pi)
amplitude = harmonicTrendCoefficients.select('sin_1').hypot(harmonicTrendCoefficients.select('cos_1')).multiply(5)
rgb = ee.Image.cat([phase, amplitude, mean_ndvi]).hsvToRgb()
resultmap = folium.Map(location=[-13.0912068,-55.9881647], zoom_start=10)
basemaps['Google Satellite'].add_to(resultmap)
latitudes = list(newdata.geometry.y.values)
longitudes = list(newdata.geometry.x.values)
labels = list(newdata['Class'].values)

for lat, lng, label in zip(latitudes, longitudes, labels):
    if label == 0:
        folium.Marker(location = [lat, lng],
        popup = str(label),
        icon = folium.Icon(color='red')
        ).add_to(resultmap)
    
    elif label == 1:
        folium.Marker(
        location = [lat, lng],
        popup = str(label),
        icon = folium.Icon(color='blue')).add_to(resultmap)
    else:
        folium.Marker(
        location = [lat, lng],
        popup = str(label),
        icon = folium.Icon(color='green')).add_to(resultmap)
        
vis_params = {'min': 0, 'max': 1}
resultmap.add_ee_layer(rgb, {}, 'phase (hue), amplitude (sat), ndvi (value)')
resultmap.add_child(folium.LayerControl())
display(resultmap)